In [2]:
!git clone https://huggingface.co/ridger/SpikeGPT-OpenWebText-216M
!git clone https://github.com/ridgerchu/SpikeGPT.git

Cloning into 'SpikeGPT-OpenWebText-216M'...
remote: Enumerating objects: 13, done.
remote: Total 13 (delta 0), reused 0 (delta 0), pack-reused 13
Unpacking objects: 100% (13/13), 1.51 KiB | 515.00 KiB/s, done.
Cloning into 'SpikeGPT'...
remote: Enumerating objects: 252, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 252 (delta 37), reused 6 (delta 1), pack-reused 187
Receiving objects: 100% (252/252), 1.28 MiB | 10.71 MiB/s, done.
Resolving deltas: 100% (94/94), done.


In [6]:
!pip install torch matplotlib numpy tqdm torchvision scipy ninja accelerate transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 87.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.3 MB/s eta 0:00:00


In [3]:
%cd SpikeGPT

/content/SpikeGPT


In [28]:
import numpy as np
import math, os, sys, types, time, gc
import torch
from src.utils import TOKENIZER
from src.model_run import RWKV_RNN
import matplotlib.ticker as ticker
from torch import nn

def load_embedding_weights(model_path):
    try:
        os.environ["CUDA_VISIBLE_DEVICES"] = sys.argv[1]
    except:
        pass
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.allow_tf32 = True
    torch.backends.cuda.matmul.allow_tf32 = True
    np.set_printoptions(precision=4, suppress=True, linewidth=200)
    args = types.SimpleNamespace()

    args.RUN_DEVICE = "cpu" # 'cuda' // 'cpu' (already fast)
    args.FLOAT_MODE = "fp32" # fp16 (good for GPU, does not work for CPU) // fp32 (good for CPU) // bf16 (less accurate, but works for CPU)
    os.environ["RWKV_JIT_ON"] = '1' # '1' or '0'. very useful for GPU/CPU fp32, but might be harmful for GPU fp16. please benchmark !!!
    vocab_size = 50277

    MODEL_NAME = model_path + 'SpikeGPT-216M'
    n_layer = 18
    n_embd = 768
    ctx_len = 1024

    args.MODEL_NAME = MODEL_NAME
    args.n_layer = n_layer
    args.n_embd = n_embd
    args.ctx_len = ctx_len
    args.vocab_size = vocab_size
    args.head_qk = 0
    args.pre_ffn = 0
    args.grad_cp = 0
    args.my_pos_emb = 0
    os.environ["RWKV_RUN_DEVICE"] = args.RUN_DEVICE

    # Load pretrained state
    model = RWKV_RNN(args)

    # Get embedding layer from the model
    emb = nn.Embedding(num_embeddings=vocab_size, embedding_dim=n_embd)
    # emb.weight.data = torch.rand(num_node,embedding_dim)
    # print(emb.weight.data)
    emb.weight.data = model.w.emb.weight.data

    return emb

def load_tokenizer():
    TOKEN_MODE = "pile"
    WORD_NAME = [
        "20B_tokenizer.json",
        "20B_tokenizer.json",
    ]  # [vocab, vocab] for Pile model
    UNKNOWN_CHAR = None

    tokenizer = TOKENIZER(WORD_NAME, UNKNOWN_CHAR=UNKNOWN_CHAR)
    if TOKEN_MODE == "pile":
        assert tokenizer.tokenizer.decode([187]) == '\n'
    return tokenizer


def transform_text_pretrained_embedding(text, emb, tokenizer):
    if tokenizer.charMode:
        context = tokenizer.refine_context(text)
        ctx = [tokenizer.stoi.get(s, tokenizer.UNKNOWN_CHAR) for s in text]
    else:
        ctx = tokenizer.tokenizer.encode(text)
    print("Number of tokens:", len(ctx))

    return emb(torch.tensor(ctx))

emb = load_embedding_weights("/content/SpikeGPT-OpenWebText-216M/")
tokenizer = load_tokenizer()
text = "In a shocking finding, scientist discovered a herd of dragons living in a remote, previously unexplored valley, in Tibet."
transform_text_pretrained_embedding(text, emb, tokenizer)

emb.weight                               float32    cpu
blocks.0.ln1.weight                      float32    cpu
blocks.0.ln1.bias                        float32    cpu
blocks.0.ln2.weight                      float32    cpu
blocks.0.ln2.bias                        float32    cpu
blocks.0.ln0.weight                      float32    cpu
blocks.0.ln0.bias                        float32    cpu
blocks.0.att.time_decay                  float32    cpu
blocks.0.att.time_first                  float32    cpu
blocks.0.att.time_mix_k                  float32    cpu
blocks.0.att.time_mix_v                  float32    cpu
blocks.0.att.time_mix_r                  float32    cpu
blocks.0.att.key.weight                  float32    cpu
blocks.0.att.value.weight                float32    cpu
blocks.0.att.receptance.weight           float32    cpu
blocks.0.att.output.weight               float32    cpu
blocks.0.ffn.time_mix_k                  float32    cpu
blocks.0.ffn.time_mix_r                  float32

tensor([[ 0.2670, -0.0613,  0.2563,  ..., -0.0652,  0.0423,  0.0334],
        [ 0.0528, -0.0115,  0.2056,  ..., -0.2686,  0.0228,  0.0199],
        [ 0.1391,  0.0236,  0.1507,  ..., -0.0139,  0.1105,  0.3136],
        ...,
        [ 0.0697, -0.0099,  0.4996,  ..., -0.1872, -0.1179,  0.0978],
        [ 0.0641, -0.0057, -0.0210,  ..., -0.0274,  0.0193,  0.0831],
        [ 0.1805, -0.0024, -0.0600,  ...,  0.3261,  0.0504, -0.1003]],
       grad_fn=<EmbeddingBackward0>)